<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

Coupled Trigger Test

This example creates 3 waves of different types, download the waves once for each DDR (depending on the number of channels in the device). Creates a task table. Download it for each channel and plays the task table according to a trigger (TRG1).triggered. And finally print registers status.

In [8]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [9]:
import os
import gc
import sys
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
#from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst

In [10]:
# Connect to instrument

# Please choose appropriate address:
inst_addr = 'TCPIP::192.168.1.24::5025::SOCKET'

inst = TEVisaInst(inst_addr)

resp = inst.send_scpi_query("*IDN?")
print('connected to: ' + resp)

connected to: Tabor Electronics,P9484M,000000220718,1.240.35 --slot#: 2, 


In [11]:
# Set the default paranoia level. 
# It controls the default behavior of: inst.send_scpi_cmd(scpi_str).
#  - paranoia-level = 0: send scpi_str as is.
#  - paranoia-level = 1: append '*OPC?' to scpi_str, and send it as query 
#  - paranoia-level = 2: append ':SYST:ERR?' to scpi_str, send it as query, and print warnning in case of error.

inst.default_paranoia_level = 2 # paranoia-level = 2 (good for debugging)

# Reset the instrument
inst.send_scpi_cmd('*CLS; *RST')
resp = inst.send_scpi_query(":SYST:ERR?")
print(resp)

0, no error


In [12]:
# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get model dependant parameters:

if model_name.startswith('P948'):
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    channels_per_dac = 2
elif model_name.startswith('P908'):
    bpp = 1
    max_dac = 255
    wpt_type = np.uint8
    channels_per_dac = 1
else:
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    channels_per_dac = 2
    
half_dac = max_dac / 2.0

# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = (int(resp) // 64) * 64
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))

max_seglen = arbmem_capacity // bpp
print ('Max segment length: {0:,}'.format(max_seglen))

Model: P9484M 
Number of channels: 4
Max segment number: 65536
Available memory per DDR: 17,171,434,432 wave-bytes
Max segment length: 8,585,717,216


In [13]:
# Build a waveform

seglen = 4096
cyclelen = seglen
ncycles = seglen / cyclelen

# sin wave:
x = np.linspace(
    start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
y = (np.sin(x) + 1.0) * half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
wav = y.astype(wpt_type)



In [14]:
# download  waveform

# Select channel
ch = 1
cmd = ':INST:CHAN {0}'.format(ch)
inst.send_scpi_cmd(cmd)

# Define segment
segnum = 1
print('Downloading segment {0} of channel {1}'.format(segnum, ch))
cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
inst.send_scpi_cmd(cmd)
               
# Select the segment
cmd = ':TRAC:SEL {0}'.format(segnum)
inst.send_scpi_cmd(cmd)

# Send the binary-data:
inst.write_binary_data(':TRAC:DATA', wav)
            
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error


In [17]:
# Play the first segment in each channel


# Select channel
cmd = ':INST:CHAN {0}'.format(segnum)
inst.send_scpi_cmd(cmd)

# Play the specified segment at the selected channel:
cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(1)
inst.send_scpi_cmd(cmd)

cmd = ':INIT:CONT OFF'
inst.send_scpi_cmd(cmd)

 # Turn on the output of the selected channel:
inst.send_scpi_cmd(':OUTP ON')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error


In [18]:
# Set up the trigger

inst.send_scpi_cmd(':TRIG:SOURce:ENAB {}'.format('TRG1'))
inst.send_scpi_cmd(':TRIG:SELECT {}'.format('TRG1'))
inst.send_scpi_cmd(':TRIG:LEV 1.0')
inst.send_scpi_cmd(':TRIG:DEL 0E-9')
inst.send_scpi_cmd(':TRIG:SLOP POS')
inst.send_scpi_cmd(':TRIG:COUP ON') # This means that trigger 1 of the master-module goes to all modules.
inst.send_scpi_cmd(':TRIGger:STAT ON')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error


In [32]:
inst.close_instrument()